In [1]:
# Cell 1 - Import required libraries and set configurations
import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

PROJECT_ROOT = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()

In [2]:
# Cell 2 - Load dataset with proper encoding
data_path = PROJECT_ROOT / 'data' / 'raw' / 'Superstore_Dataset.csv'
try:
    df = pd.read_csv(data_path, parse_dates=['Order Date', 'Ship Date'], encoding='windows-1252')
    print('Dataset loaded successfully.')
except FileNotFoundError:
    print(f'Error: {data_path} not found. Ensure the dataset is in data/raw/.')
    raise
except UnicodeDecodeError:
    print(f'Error: Unable to decode {data_path} with windows-1252. Trying latin1...')
    df = pd.read_csv(data_path, parse_dates=['Order Date', 'Ship Date'], encoding='latin1')
    print('Dataset loaded successfully with latin1 encoding.')

print('\nInitial Dataset Info:')
df.info()

Dataset loaded successfully.

Initial Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Row ID         9994 non-null   int64         
 1   Order ID       9994 non-null   object        
 2   Order Date     9994 non-null   datetime64[ns]
 3   Ship Date      9994 non-null   datetime64[ns]
 4   Ship Mode      9994 non-null   object        
 5   Customer ID    9994 non-null   object        
 6   Customer Name  9994 non-null   object        
 7   Segment        9994 non-null   object        
 8   Country        9994 non-null   object        
 9   City           9994 non-null   object        
 10  State          9994 non-null   object        
 11  Postal Code    9994 non-null   int64         
 12  Region         9994 non-null   object        
 13  Product ID     9994 non-null   object        
 14  Category       9994 

In [3]:
# Cell 3 - Check for missing values and duplicates
print('\nMissing Values by Column:')
print(df.isnull().sum())

print('\nNumber of Duplicate Rows:')
print(df.duplicated().sum())

if df.duplicated().sum() > 0:
    df = df.drop_duplicates()
    print(f'Duplicates removed. New row count: {len(df)}')


Missing Values by Column:
Row ID           0
Order ID         0
Order Date       0
Ship Date        0
Ship Mode        0
Customer ID      0
Customer Name    0
Segment          0
Country          0
City             0
State            0
Postal Code      0
Region           0
Product ID       0
Category         0
Sub-Category     0
Product Name     0
Sales            0
Quantity         0
Discount         0
Profit           0
dtype: int64

Number of Duplicate Rows:
0


In [4]:
# Cell 4 - Handle inconsistent data types and outliers
numerical_cols = ['Sales', 'Quantity', 'Discount', 'Profit']

for col in numerical_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

print('\nOutliers Detection (IQR Method):')
for col in numerical_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)][col]
    print(f'{col} - Outliers count: {len(outliers)}')

df['Quantity'] = df['Quantity'].clip(lower=0)
df['Sales'] = df['Sales'].clip(lower=0)
df['Profit'] = df['Profit'].fillna(df['Profit'].median())


Outliers Detection (IQR Method):
Sales - Outliers count: 1167
Quantity - Outliers count: 170
Discount - Outliers count: 856
Profit - Outliers count: 1881


In [5]:
# Cell 5 - Standardize categorical variables
categorical_cols = ['Ship Mode', 'Segment', 'Country', 'Region', 'Category', 'Sub-Category']

for col in categorical_cols:
    df[col] = df[col].str.strip().str.title()

print('\nUnique Values After Standardization:')
for col in categorical_cols:
    print(f'{col}: {df[col].unique()}')


Unique Values After Standardization:
Ship Mode: ['Second Class' 'Standard Class' 'First Class' 'Same Day']
Segment: ['Consumer' 'Corporate' 'Home Office']
Country: ['United States']
Region: ['South' 'West' 'Central' 'East']
Category: ['Furniture' 'Office Supplies' 'Technology']
Sub-Category: ['Bookcases' 'Chairs' 'Labels' 'Tables' 'Storage' 'Furnishings' 'Art'
 'Phones' 'Binders' 'Appliances' 'Paper' 'Accessories' 'Envelopes'
 'Fasteners' 'Supplies' 'Machines' 'Copiers']


In [6]:
# Cell 6 - Encode categorical variables for modeling
df_encoded = df.copy()

for col in categorical_cols:
    df_encoded = pd.concat([df_encoded, pd.get_dummies(df_encoded[col], prefix=col, drop_first=True)], axis=1)
    df_encoded.drop(col, axis=1, inplace=True)

print('\nEncoded Dataset Info:')
df_encoded.info()


Encoded Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 41 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Row ID                    9994 non-null   int64         
 1   Order ID                  9994 non-null   object        
 2   Order Date                9994 non-null   datetime64[ns]
 3   Ship Date                 9994 non-null   datetime64[ns]
 4   Customer ID               9994 non-null   object        
 5   Customer Name             9994 non-null   object        
 6   City                      9994 non-null   object        
 7   State                     9994 non-null   object        
 8   Postal Code               9994 non-null   int64         
 9   Product ID                9994 non-null   object        
 10  Product Name              9994 non-null   object        
 11  Sales                     9994 non-null   float64       
 1

In [7]:
# Cell 7 - Feature engineering with date columns
df_encoded['Order Year'] = df_encoded['Order Date'].dt.year
df_encoded['Order Month'] = df_encoded['Order Date'].dt.month
df_encoded['Ship Days'] = (df_encoded['Ship Date'] - df_encoded['Order Date']).dt.days

df_encoded.drop(['Order Date', 'Ship Date'], axis=1, inplace=True)

print('\nDataset Info After Feature Engineering:')
df_encoded.info()


Dataset Info After Feature Engineering:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 42 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Row ID                    9994 non-null   int64  
 1   Order ID                  9994 non-null   object 
 2   Customer ID               9994 non-null   object 
 3   Customer Name             9994 non-null   object 
 4   City                      9994 non-null   object 
 5   State                     9994 non-null   object 
 6   Postal Code               9994 non-null   int64  
 7   Product ID                9994 non-null   object 
 8   Product Name              9994 non-null   object 
 9   Sales                     9994 non-null   float64
 10  Quantity                  9994 non-null   int64  
 11  Discount                  9994 non-null   float64
 12  Profit                    9994 non-null   float64
 13  Ship Mode_Same Day    

In [8]:
# Cell 8 - Save cleaned dataset
output_path = PROJECT_ROOT / 'data' / 'processed' / 'cleaned_superstore_data.csv'
df_encoded.to_csv(output_path, index=False)
print(f'Cleaned dataset saved to: {output_path}')

Cleaned dataset saved to: /Users/rashidkarimov/Desktop/supestore_project/data/processed/cleaned_superstore_data.csv
